# Loading Games and Teams
This will load all the games and teams and process them ready to be analyzed.

In [56]:
import os
import requests
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from sklearn.preprocessing import StandardScaler

from functools import reduce

load_dotenv()
league_id = os.getenv('league_id')
current_year = int(os.getenv('current_year'))
url = "https://fantasy.espn.com/apis/v3/games/ffl/seasons/{}/segments/0/leagues/{}".format(current_year, league_id)
history_url = "https://fantasy.espn.com/apis/v3/games/ffl/leagueHistory/{}".format(league_id)
cookies = {
        "swid": os.getenv('swid'),
        "espn_s2": os.getenv('espn_s2')
    }

## Load All the Games

In [57]:
#for games
def add_year(game, season_id):
    game["year"] = season_id
    return game

def get_schedule(season_id):
    hist_req = requests.get(history_url,
        cookies=cookies,
        params={"view": "mMatchup", "seasonId": season_id,})
    hist_list = list(map(lambda x: add_year(x, season_id), hist_req.json()[0]["schedule"]))
    return hist_list
    
games_req = requests.get(url,
params={"view": "mMatchup"},
    cookies=cookies)
schedule = list(map(lambda x: add_year(x, current_year), games_req.json()["schedule"]))

# load all the previous seasons from the response
prev = games_req.json()["status"]["previousSeasons"]
# builds the full schedule for each year
prev_games = list(map(get_schedule, prev))

# flatten all the historical games to the current list of games
all_games = schedule + reduce(list.__add__, prev_games)

# process everything to just the weeks, teams, scores
games_df = pd.DataFrame([[game["matchupPeriodId"], 
game["home"]["teamId"], game["home"]["totalPoints"],
game["away"]["teamId"], game["away"]["totalPoints"], game["year"]] for game in all_games],
    columns=['Week', 'Team1', 'Score1', 'Team2', 'Score2', "year"])

# calculate win margins
game_margin_df = games_df.assign(Margin1 = games_df["Score1"] - games_df["Score2"],
    Margin2 = games_df["Score2"] - games_df["Score1"])

# rename and save
g_flattened = (game_margin_df[['Week', 'Team1', 'Margin1', 'Score1', 'Score2', 'year']]
    .rename(columns={'Team1': 'Team', 'Score1': 'Score', 'Margin1': 'Margin', 'Score2': 'OppScore'})
    .append(game_margin_df[['Week', 'Team2', 'Margin2', 'Score2', 'Score1', 'year']]
    .rename(columns={'Team2': 'Team', 'Score2': 'Score', 'Margin2': 'Margin', 'Score1': 'OppScore'})
    ))

# get the mean for each week
def add_week_avg(grp):
    grp["week_mean"] = grp["Score"].mean()
    return grp

# apply the mean
g_flattened = g_flattened.groupby(["year", "Week"]).apply(add_week_avg)

# add wins and diff from mean
g_flattened["win"] = g_flattened["Margin"] > 0
g_flattened["for_diff"] = g_flattened["Score"] - g_flattened["week_mean"]
g_flattened["against_diff"] = g_flattened["OppScore"] - g_flattened["week_mean"]

# calculate unlucky and lucky
g_flattened["unlucky_loss"] = 0
g_flattened["lucky_win"] = 0
g_flattened.loc[(g_flattened["win"] == False) & (g_flattened["for_diff"] > 0), "unlucky_loss"] = 1
g_flattened.loc[(g_flattened["win"] == True) & (g_flattened["for_diff"] < 0), "lucky_win"] = 1


## Load All the Teams

In [58]:
#for teams
def get_data(args):
    season_id, load_url = args
    team_req = requests.get(load_url,
        cookies=cookies,
        params={"seasonId": season_id, "view": "mTeam"})
    teams = team_req.json()[0]["teams"]
    teams_df = pd.DataFrame(pd.json_normalize(teams))
    teams_df["year"] = season_id
    return teams_df
    
current_teams_req = requests.get(url,
    cookies=cookies,
    params={"view": "mTeam"})
curr_teams = current_teams_req.json()["teams"]
year = current_teams_req.json()["seasonId"]
full_teams_df = pd.DataFrame(pd.json_normalize(curr_teams))
full_teams_df["year"] = year

# build the call to load all the previous teams
mapped_hist_teams = map(get_data, list(map(lambda y: (y, history_url), prev)))
full_teams_df = full_teams_df.append(pd.concat(list(mapped_hist_teams)))

# scale the data to compare between years
scaler = StandardScaler()
def zscore(group):
    scaled = scaler.fit_transform(X=group[["record.overall.pointsFor", "record.overall.pointsAgainst", 'transactionCounter.moveToActive']])
    group["scaled_pointsFor"], group["scaled_pointsAgainst"], group["scaled_moves"] = scaled[:,0], scaled[:,1], scaled[:,2]
    return group

full_scaled_df = full_teams_df.groupby("year").apply(zscore)

# logic to determin in the money (itm) or last place
def calc_itm_last(group):
    group.loc[group["rankCalculatedFinal"] <= 3, 'itm'] = 1
    group.loc[group["rankCalculatedFinal"] > 3, 'itm'] = 0
    group.loc[group["rankCalculatedFinal"] == 0, 'itm'] = np.nan
    group.loc[(group["rankCalculatedFinal"] == group["rankCalculatedFinal"].max()) & (group["rankCalculatedFinal"] != 0), 'last_place'] = 1
    group.loc[group["last_place"] != 1, 'last_place'] = 0
    return group

itm_teams = full_scaled_df.groupby("year").apply(calc_itm_last)

itm_teams.to_csv("processed_teams.csv")

## Add the Team Abbreviation

In [59]:
abbrev = itm_teams[['abbrev', 'year', 'id']]
games_abbrev = g_flattened.merge(abbrev, left_on=['year', 'Team'], right_on=['year', 'id'])
games_abbrev.to_csv("processed_games.csv")